In [ ]:
from llama_index.llms.google_genai import GoogleGenAI

llm = GoogleGenAI(model="gemini-2.0-flash")
resp = llm.complete("Write a poem about a magic backpack")
print(resp)

In [ ]:
from llama_index.core.llms import ChatMessage
from llama_index.llms.google_genai import GoogleGenAI

messages = [
    ChatMessage(role="user", content="Hello friend!"),
    ChatMessage(role="assistant", content="Yarr what is shakin' matey?"),
    ChatMessage(
        role="user", content="Help me decide what to have for dinner."
    ),
]

llm = GoogleGenAI(model="gemini-2.0-flash")
# resp = llm.chat(messages)
# print(resp)

print("Streaming response:")
for chunk in llm.stream_chat(messages):
    print(chunk, end="", flush=True)

In [8]:
from datasets import load_dataset
from pathlib import Path

dataset = load_dataset(path="dvilasuero/finepersonas-v0.1-tiny", split="train")

Path("data").mkdir(parents=True, exist_ok=True)
for i, persona in enumerate(dataset):
    with open(Path("data") / f"persona_{i}.txt", "w") as f:
        f.write(persona["persona"])

README.md:   0%|          | 0.00/618 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


train-00000-of-00001.parquet:   0%|          | 0.00/35.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [9]:
from llama_index.core import SimpleDirectoryReader

reader = SimpleDirectoryReader(input_dir="data")
documents = reader.load_data()
len(documents)

5000

In [10]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import Document
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline

db = chromadb.PersistentClient(path="./alfred_chroma_db")
chroma_collection = db.get_or_create_collection(name="alfred")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(),
        HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5"),
    ],
    vector_store=vector_store,
)

nodes = await pipeline.arun(documents=documents[:10])
len(nodes)

10

In [11]:
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
index = VectorStoreIndex.from_vector_store(vector_store, embed_model=embed_model)

In [14]:
from llama_index.llms.google_genai import GoogleGenAI

llm = GoogleGenAI(model="gemini-2.0-flash")
query_engine = index.as_query_engine(
    llm=llm,
    response_mode="tree_summarize",
)
response = query_engine.query("Respond using a persona that describes author and travel experiences?")
print(response)

I am either an anthropologist or cultural expert immersed in Cypriot culture, history, and society, or a pulmonologist passionate about educating patients about the respiratory system and its diseases. My travels have either been spent researching and living in Cyprus, or perhaps they are related to my work as a respiratory specialist.



In [1]:
from llama_index.core import SimpleDirectoryReader

reader = SimpleDirectoryReader(input_dir="C:\\Users\\IdeaPad\\Documents\\project\\LLM Book\\Alammar J.. Hands-On Large Language Models...6ed 2024 Eariy Release")
documents = reader.load_data()
len(documents)

383

In [7]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import Document
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline

db = chromadb.PersistentClient(path="./book_chroma_db")
chroma_collection = db.get_or_create_collection(name="book")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(),
        HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5"),
    ],
    vector_store=vector_store,
)

nodes = await pipeline.arun(documents=documents)
len(nodes)

383

In [8]:
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
index = VectorStoreIndex.from_vector_store(vector_store, embed_model=embed_model)


In [9]:
from llama_index.llms.google_genai import GoogleGenAI

llm = GoogleGenAI(model="gemini-2.0-flash")
query_engine = index.as_query_engine(
    llm=llm,
    response_mode="tree_summarize",
)
response = query_engine.query("What is the main idea of the book?")
print(response)

The key aspects of effectively using LLMs involve: clearly defining your objectives with specificity, understanding that LLMs may generate incorrect information, strategically placing instructions at the beginning or end of prompts, defining the role the LLM should adopt, providing a specific task, giving additional context, specifying the desired output format, identifying the target audience, and setting the tone for the generated text.



In [11]:
response = query_engine.query("What is one of the most common tasks in natural language processing?")
print(response)

One of the most representative words for a topic seems to be summarization.



In [ ]:
from llama_index.core.evaluation import FaithfulnessEvaluator

# query_engine = # from the previous section
# llm = # from the previous section

# query index
evaluator = FaithfulnessEvaluator(llm=llm)
response = query_engine.query(
    "What battles took place in New York City in the American Revolution?"
)
eval_result = evaluator.evaluate_response(response=response)
eval_result.passing

In [14]:
evaluator = FaithfulnessEvaluator(llm=llm)
response = query_engine.query(
    "What is one of the most common tasks in natural language processing?"
)
eval_result = evaluator.evaluate_response(response=response)
eval_result.passing

True